# Практическая работа №4: Элементы корреляционного анализа. Проверка статистической гипотезы о равенстве коэффициента корреляции нулю

Выполнили студенты гр. 0383 Желнин Максим, Рудакова Юлия, Сергевнин Дмитрий. Вариант №12


## Цель работы
Освоение основных понятий, связанных с корреляционной зависимостью между случайными величинами, статистическими гипотезами и проверкой их «справедливости».

## Основные теоретические положения

*Функциональная зависимость* — это такая зависимость между двумя
переменными, при которой каждому значению независимой переменной
соответствует единственное значение зависимой переменной.


*Статистическая зависимость* — это связь двух случайных величин, при
которой распределение вероятностей одной из них зависит от того, какие
возможные значения приняла другая величина.


*Корреляционная зависимость* – это согласованные изменения двух (парная
корреляционная связь) или большего количества признаков (множественная
корреляционная связь). Суть ее заключается в том, что при изменении значения
одной переменной происходит закономерное изменение другой(-их)
переменной(-ых).

*Коэффициент корреляции* – двумерная описательная статистика,
количественная мера взаимосвязи (совместной изменчивости) двух переменных


Свойства коэффициента корреляции:
* Если если $𝜉$ и $𝜂$ независимы, то $𝑝(𝜉 , 𝜂) = 0$
* Всегда $|𝑝(𝜉 , 𝜂)| ≤ 1$
* $𝑝(𝜉 , 𝜂)| = 1$ тогда и только тогда, когда $𝜉$ и $𝜂$ линейно связаны.

*Интервальный вариационный ряд*  – это
упорядоченное множество смежных интервалов и соответствующие им частоты,
в сумме равные объёму совокупности.

Выборочный коэффициент корреляции, стандартная формула:

$𝑟_{𝑥y} = \frac{\sum_ {i=1} ^{n}(𝑥_i − \bar{x})(𝑦_i−\bar{y})}{\sqrt{\sum_ {i=1} ^{n}(𝑥_i − \bar{x})^2} * \sqrt{\sum_ {i=1} ^{n}(y_i − \bar{y})^2}}$


Формула для выборочных условных:

$𝑟_{𝑥y} = \frac{\sum_ {i=1} ^{K_y}\sum_ {i=1} ^{K_x}(n_{ij}u_i v_j - N \bar{u}_B \bar{v}_B)}{N \bar{\sigma}_u \bar{\sigma}_v}$, где $u_i = \frac{x_i - C_x}{h_x}$, $v_j = \frac{y_j - C_y}{h_y}$, $\bar{\sigma}_n = \sqrt{u^\bar{2} - \bar{u}^2}$, $\bar{\sigma}_v = \sqrt{v^\bar{2} - \bar{v}^2}$


Формула для расчета доверительного интервала:

$z \in  (z_B - λ(\gamma)\sigma_z; z_B + λ(\gamma)\sigma_z)$, где $z_B = \frac{1}{2} ln(\frac{1 + r_B}{1 - r_B})$, $\sigma_z = \frac{1}{\sqrt(N-3} $

## Постановка задачи

Из заданной генеральной совокупности сформировать выборку по второму признаку. Провести статистическую обработку второй выборки в объёме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса и коэффициента вариации). Для системы двух случайных величин $X$ (первый признак) и $Y$ (второй признак) сформировать двумерную выборку и найти статистическую оценку коэффициента корреляции, построить доверительный интервал для коэффициента корреляции и осуществить проверку статистической гипотезы о равенстве коэффициента корреляции нулю. Полученные результаты содержательно проинтерпретировать.

## Выполнение работы


### 1.
Провести статистическую обработку второй выборки в объеме практических работ №1 и №2, с целью определения точечных статистических оценок параметров распределения исследуемого признака (математического ожидания, дисперсии, среднеквадратичного отклонения, асимметрии, эксцесса, моды, медианы и коэффициента вариации). Оформить результаты в виде таблицы, сделать выводы.

In [ ]:
import pandas as pd
import numpy as np
import io
import zipfile
from google.colab import files
import math
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt

In [ ]:
uploaded = files.upload()

Saving sample.zip to sample.zip


In [ ]:
with zipfile.ZipFile(io.BytesIO(uploaded['sample.zip']), 'r') as zip_ref:
    zip_ref.extractall()

df = pd.read_csv('sample.csv', skiprows=3) # двумерная генеральная совокупность
N = 111
sample = df.sample(n=N, random_state=1)
sample

,nu,E
398,476,143.0
125,541,146.8
328,428,113.7
339,500,136.6
172,372,81.7
...,...,...
73,390,108.1
233,474,122.6
298,437,115.1
250,434,110.5


In [ ]:
ranked_nu = sample.sort_values(by='nu').reset_index(drop=True)['nu']
ranked_e = sample.sort_values(by='E').reset_index(drop=True)['E']

In [ ]:
# Вычисляем среднеквадратичное отклонение
std_dev_nu = np.std(ranked_nu)
std_dev_e = np.std(ranked_e)

print("Среднеквадратичное отклонение для выборки ranked_nu:", std_dev_nu)
print("Среднеквадратичное отклонение для выборки ranked_e:", std_dev_e)

Среднеквадратичное отклонение для выборки ranked_nu: 60.57321887248774
Среднеквадратичное отклонение для выборки ranked_e: 24.246649855565774


In [ ]:
# Преобразование ранжированного ряда в вариационный ряд для 'nu'
variation_nu = ranked_nu.value_counts().reset_index()
variation_nu.columns = ['значение', 'частота']
variation_nu_sort = variation_nu.sort_values(by='значение').reset_index(drop=True)
variation_nu_sort.drop(columns=['частота'], inplace=True)


# Преобразование ранжированного ряда в вариационный ряд для 'E'
variation_e = ranked_e.value_counts().reset_index()
variation_e.columns = ['значение', 'частота']
variation_e_sort = variation_e.sort_values(by='значение').reset_index(drop=True)
variation_e_sort.drop(columns=['частота'], inplace=True)

In [ ]:
def create_interval_series(variation_series):
    # Определение интервалов
    num_bins = round(1 + 3.31 * math.log10(N))
    if num_bins % 2 == 0:
      num_bins += 1

    bin_width = (variation_series['значение'].max() - variation_series['значение'].min()) / num_bins
    bins = [variation_series['значение'].min() + i * bin_width for i in range(num_bins)]
    bins.append(variation_series['значение'].max())

    # Подсчет частоты попадания в интервалы
    frequency, _ = np.histogram(variation_series['значение'], bins=bins)

    # Определение границ и середин интервалов
    interval_bounds = [(bins[i], bins[i+1]) for i in range(num_bins)]
    interval_midpoints = [(interval[0] + interval[1]) / 2 for interval in interval_bounds]

    interval_series = pd.DataFrame({
        'Границы интервалов': interval_bounds,
        'Середины интервалов': interval_midpoints,
        'Частота попадания в интервал': frequency,
        'Относительная частота': frequency / len(variation_series)
    })

    return interval_series


interval_nu_series = create_interval_series(variation_nu_sort)
interval_e_series = create_interval_series(variation_e_sort)
print(interval_e_series['Относительная частота'])

0    0.028302
1    0.094340
2    0.122642
3    0.122642
4    0.179245
5    0.169811
6    0.160377
7    0.066038
8    0.056604
Name: Относительная частота, dtype: float64


In [ ]:
def get_buckets(intervals, series):
    buckets = [[] for i in range(len(intervals) - 1)]
    for i in range(len(series) - 1):
      for j in range(len(intervals) - 1):
        if series[i] > intervals[j][0] and series[i] <= intervals[j][1]:
          buckets[j].append(series[i])
    return buckets

Для NU

In [ ]:
mid_nu_values = interval_nu_series["Середины интервалов"]

buckets_nu = get_buckets(interval_nu_series["Границы интервалов"], variation_nu["значение"])
C_nu = mid_nu_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_nu)])[1]]
h_nu = int((mid_nu_values[len(mid_nu_values) - 1] - mid_nu_values[0]) /(len(mid_nu_values)-1))
ui_arr_nu = [round((xi - C_nu) / h_nu) for xi in mid_nu_values]
nu_freq_arr = interval_nu_series["Частота попадания в интервал"]
len_intervals_nu = len(ui_arr_nu)

data_nu = {'xi': mid_nu_values,
        'ni': nu_freq_arr,
        'ui':  ui_arr_nu,
        'ui*n': [nu_freq_arr[i] * ui_arr_nu[i] for i in range(len_intervals_nu)],
        'ui^2 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 2) for i in range(len_intervals_nu)],
        'ui^3 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 3) for i in range(len_intervals_nu)],
        'ui^4 * n': [nu_freq_arr[i] * (ui_arr_nu[i] ** 4) for i in range(len_intervals_nu)],
        '(ui + 1)^4 * n': [nu_freq_arr[i] * ((ui_arr_nu[i] + 1) ** 4) for i in range(len_intervals_nu)]}

table1_nu = pd.DataFrame(data_nu)
table1_nu.loc['Sum'] = table1_nu.sum()
print("Таблица 1:")
print(table1_nu)

Таблица 1:
              xi    ni   ui  ui*n  ui^2 * n  ui^3 * n  ui^4 * n  \
0     335.055556   3.0 -5.0 -15.0      75.0    -375.0    1875.0   
1     365.166667   8.0 -4.0 -32.0     128.0    -512.0    2048.0   
2     395.277778  13.0 -3.0 -39.0     117.0    -351.0    1053.0   
3     425.388889  12.0 -2.0 -24.0      48.0     -96.0     192.0   
4     455.500000  13.0 -1.0 -13.0      13.0     -13.0      13.0   
5     485.611111  15.0  0.0   0.0       0.0       0.0       0.0   
6     515.722222  11.0  1.0  11.0      11.0      11.0      11.0   
7     545.833333   8.0  2.0  16.0      32.0      64.0     128.0   
8     575.944444   3.0  3.0   9.0      27.0      81.0     243.0   
Sum  4099.500000  86.0 -9.0 -87.0     451.0   -1191.0    5563.0   

     (ui + 1)^4 * n  
0             768.0  
1             648.0  
2             208.0  
3              12.0  
4               0.0  
5              15.0  
6             176.0  
7             648.0  
8             768.0  
Sum          3243.0  


In [ ]:
nu_N = sum(data_nu["ni"])
vi_nu_arr = [sum(data_nu["ui*n"]) / nu_N, sum(data_nu["ui^2 * n"]) / nu_N, sum(data_nu["ui^3 * n"]) / nu_N, sum(data_nu["ui^4 * n"]) / nu_N]
mi_nu_arr = [0, (vi_nu_arr[1] - vi_nu_arr[0] ** 2) * (h_nu ** 2), (vi_nu_arr[2] - 3* vi_nu_arr[1] * vi_nu_arr[0]  + 2 * (vi_nu_arr[0]** 3)) * (h_nu ** 3), (vi_nu_arr[3] - 4 * vi_nu_arr[2] * vi_nu_arr[0] + 6 * vi_nu_arr[1] * (vi_nu_arr[0] ** 2)  - 3 * (vi_nu_arr[0]** 4)) * (h_nu ** 4)]

In [ ]:
x_v2_nu = sum(ranked_nu)/N

sigma2_nu = sum([(ranked_nu[i] - x_v2_nu) ** 2 for i in range(N)]) / N
print("Мат. ожидание: ", x_v2_nu)
print("Дисперсия: ", sigma2_nu)

Мат. ожидание:  454.1711711711712
Дисперсия:  3669.114844574306


Исправленная выборочная дисперсия $S^{2}$ и исправленное СКО S.


In [ ]:
S_square = sigma2_nu * N / (N-1)
S_nu = np.sqrt(S_square)
print(S_square, S_nu)

3702.470434070436 60.847928757439526


In [ ]:
a_s = mi_nu_arr[2] / (S_nu ** 3)
e_s = mi_nu_arr[3] / (S_nu ** 4) - 3
print(a_s, e_s)

-0.00047030066484368395 -0.7720328739418365


In [ ]:
moda = data_nu["xi"][3] + h_nu * (interval_nu_series['Относительная частота'][4]-interval_nu_series['Относительная частота'][3])/(interval_nu_series['Относительная частота'][4]-interval_nu_series['Относительная частота'][3] + interval_nu_series['Относительная частота'][4]-interval_nu_series['Относительная частота'][5])
print(moda)

395.38888888888897


In [ ]:
print(x_v2_nu / sigma2_nu * 100)

12.378221734944482


Для Е

In [ ]:
mid_e_values = interval_e_series["Середины интервалов"]

buckets_e = get_buckets(interval_e_series["Границы интервалов"], variation_e["значение"])
C_e = mid_e_values[max([(len(bucket), i) for i, bucket in enumerate(buckets_e)])[1]]
h_e = int((mid_e_values[len(mid_e_values) - 1] - mid_e_values[0]) /(len(mid_e_values)-1))
ui_arr_e = [(xi - C_e) / h_e for xi in mid_e_values]
e_freq_arr = interval_e_series["Частота попадания в интервал"]
len_intervals_e = len(ui_arr_e)

data_e = {'xi': mid_e_values,
        'ni': e_freq_arr,
        'ui':  ui_arr_e,
        'ui*n': [e_freq_arr[i] * ui_arr_e[i] for i in range(len_intervals_e)],
        'ui^2 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 2) for i in range(len_intervals_e)],
        'ui^3 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 3) for i in range(len_intervals_e)],
        'ui^4 * n': [e_freq_arr[i] * (ui_arr_e[i] ** 4) for i in range(len_intervals_e)],
        '(ui + 1)^4 * n': [e_freq_arr[i] * ((ui_arr_e[i] + 1) ** 4) for i in range(len_intervals_e)]}

table1_e = pd.DataFrame(data_e)
table1_e.loc['Sum'] = table1_e.sum()
print("Таблица 1:")
print(table1_e)

Таблица 1:
              xi     ni            ui       ui*n    ui^2 * n    ui^3 * n  \
0      78.622222    3.0 -4.014815e+00 -12.044444   48.356214 -194.141244   
1      90.666667   10.0 -3.011111e+00 -30.111111   90.667901 -273.011125   
2     102.711111   13.0 -2.007407e+00 -26.096296   52.385898 -105.159841   
3     114.755556   13.0 -1.003704e+00 -13.048148   13.096475  -13.144980   
4     126.800000   19.0  0.000000e+00   0.000000    0.000000    0.000000   
5     138.844444   18.0  1.003704e+00  18.066667   18.133580   18.200742   
6     150.888889   17.0  2.007407e+00  34.125926   68.504636  137.516715   
7     162.933333    7.0  3.011111e+00  21.077778   63.467531  191.107787   
8     174.977778    6.0  4.014815e+00  24.088889   96.712428  388.282489   
Sum  1141.200000  106.0 -2.664535e-15  16.059259  451.324664  149.650542   

        ui^4 * n  (ui + 1)^4 * n  
0     779.441144    2.478357e+02  
1     822.066831    1.635853e+02  
2     211.098643    1.338949e+01  
3      13.19

In [ ]:
vi_e_arr = [sum(data_e["ui*n"]) / N, sum(data_e["ui^2 * n"]) / N, sum(data_e["ui^3 * n"]) / N, sum(data_e["ui^4 * n"]) / N]
mi_e_arr = [0, (vi_e_arr[1] - vi_e_arr[0] ** 2) * (h_e ** 2), (vi_e_arr[2] - 3* vi_e_arr[1] * vi_e_arr[0]  + 2 * (vi_e_arr[0]** 3)) * (h_e ** 3), (vi_e_arr[3] - 4 * vi_e_arr[2] * vi_e_arr[0] + 6 * vi_e_arr[1] * (vi_e_arr[0] ** 2)  - 3 * (vi_e_arr[0]** 4)) * (h_e ** 4)]


In [ ]:
x_v2_e = sum(ranked_e)/N

sigma2_e = sum([(ranked_e[i] - x_v2_e) ** 2 for i in range(N)]) / N
print("Мат. ожидание: ", x_v2_e)
print("Дисперсия: ", sigma2_e)

Мат. ожидание:  128.63243243243238
Дисперсия:  587.9000292184074


Исправленная выборочная дисперсия $S^{2}$ и исправленное СКО S.




In [ ]:
S_square = sigma2_e * N / (N-1)
S_e = np.sqrt(S_square)
print(S_square, S_e)

593.2445749385748 24.35661255056981


In [ ]:
a_s = mi_e_arr[2] / (S_e ** 3)
e_s = mi_e_arr[3] / (S_e ** 4) - 3
print(a_s, e_s)

-0.049093567588123006 -0.7576802147789534


In [ ]:
moda = data_e["xi"][3] + h_e * (interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][3])/(interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][3] + interval_e_series['Относительная частота'][4]-interval_e_series['Относительная частота'][5])
print(moda)

125.04126984126984


In [ ]:
print(x_v2_e / sigma2_e * 100)

21.879984017596446


2.
Построить двумерный интервальный вариационный ряд, оформить в виде таблицы.

In [ ]:
interval_nu = [str(interval) for interval in interval_nu_series['Границы интервалов']]
for i,e in enumerate(interval_nu):
  interval_nu[i] = e.replace("(", "[")
  if i == len(interval_nu) - 1:
    interval_nu[i] = e.replace(")", "]").replace("(", "[")

interval_e = [str(interval) for interval in interval_e_series['Границы интервалов']]
for i,e in enumerate(interval_e):
  interval_e[i] = e.replace("(", "[")
  if i == len(interval_e) - 1:
    interval_e[i] = e.replace(")", "]").replace("(", "[")
print(interval_nu)
print(interval_e)

['[320.0, 350.1111111111111)', '[350.1111111111111, 380.22222222222223)', '[380.22222222222223, 410.3333333333333)', '[410.3333333333333, 440.44444444444446)', '[440.44444444444446, 470.55555555555554)', '[470.55555555555554, 500.66666666666663)', '[500.66666666666663, 530.7777777777778)', '[530.7777777777778, 560.8888888888889)', '[560.8888888888889, 591]']
['[72.6, 84.64444444444445)', '[84.64444444444445, 96.68888888888888)', '[96.68888888888888, 108.73333333333332)', '[108.73333333333332, 120.77777777777777)', '[120.77777777777777, 132.82222222222222)', '[132.82222222222222, 144.86666666666667)', '[144.86666666666667, 156.9111111111111)', '[156.9111111111111, 168.95555555555555)', '[168.95555555555555, 181.0]']


In [ ]:
_N = N -1
two_dimensional_variational_series = pd.DataFrame(index=interval_nu, columns=interval_e)
two_dimensional_variational_series.fillna(0, inplace=True)

for _, row in sample.iterrows():
    nu_interval = None
    E_interval = None

    # Находим интервалы, в которые попадают данные из sample для nu и E
    for index, interval in enumerate(interval_nu_series['Границы интервалов']):
        if interval[0] <= row['nu'] < interval[1]:
            nu_interval = interval_nu[index]
            break
        elif index == len(interval_nu_series['Границы интервалов']) - 1:
            nu_interval = interval_nu[index]

    for index, interval in enumerate(interval_e_series['Границы интервалов']):
        if interval[0] <= row['E'] < interval[1]:
            E_interval = interval_e[index]
            break
        elif index == len(interval_e_series['Границы интервалов']) - 1:
            nu_interval = interval_e[index]
    if nu_interval is not None and E_interval is not None:
        two_dimensional_variational_series[E_interval][nu_interval] += 1

print("Двумерный интервальный вариационный ряд:")
two_dimensional_variational_series


Двумерный интервальный вариационный ряд:


,"[72.6, 84.64444444444445)","[84.64444444444445, 96.68888888888888)","[96.68888888888888, 108.73333333333332)","[108.73333333333332, 120.77777777777777)","[120.77777777777777, 132.82222222222222)","[132.82222222222222, 144.86666666666667)","[144.86666666666667, 156.9111111111111)","[156.9111111111111, 168.95555555555555)","[168.95555555555555, 181.0]"
"[320.0, 350.1111111111111)",1,2,0,0,0,0,0,0,0
"[350.1111111111111, 380.22222222222223)",2,5,3,0,0,0,0,0,0
"[380.22222222222223, 410.3333333333333)",0,3,7,5,1,0,0,0,0
"[410.3333333333333, 440.44444444444446)",0,0,2,9,7,1,0,0,0
"[440.44444444444446, 470.55555555555554)",0,0,1,1,8,7,0,0,0
"[470.55555555555554, 500.66666666666663)",0,0,0,0,3,7,8,0,0
"[500.66666666666663, 530.7777777777778)",0,0,0,0,1,5,4,4,1
"[530.7777777777778, 560.8888888888889)",0,0,0,0,0,0,4,2,3
"[560.8888888888889, 591]",0,0,0,0,0,0,1,1,1


3.
По полученному двумерному интервальному вариационному ряду построить корреляционную таблицу.

In [ ]:
two_dimensional_variational_series['Сумма'] = two_dimensional_variational_series.sum(axis=1)

In [ ]:
new_row = [two_dimensional_variational_series[col].sum() for col in two_dimensional_variational_series.columns]
new_row

[3, 10, 13, 15, 20, 20, 17, 7, 5, 110]

In [ ]:
two_dimensional_variational_series.loc['Сумма'] = new_row
two_dimensional_variational_series

,"[72.6, 84.64444444444445)","[84.64444444444445, 96.68888888888888)","[96.68888888888888, 108.73333333333332)","[108.73333333333332, 120.77777777777777)","[120.77777777777777, 132.82222222222222)","[132.82222222222222, 144.86666666666667)","[144.86666666666667, 156.9111111111111)","[156.9111111111111, 168.95555555555555)","[168.95555555555555, 181.0]",Сумма
"[320.0, 350.1111111111111)",1,2,0,0,0,0,0,0,0,3
"[350.1111111111111, 380.22222222222223)",2,5,3,0,0,0,0,0,0,10
"[380.22222222222223, 410.3333333333333)",0,3,7,5,1,0,0,0,0,16
"[410.3333333333333, 440.44444444444446)",0,0,2,9,7,1,0,0,0,19
"[440.44444444444446, 470.55555555555554)",0,0,1,1,8,7,0,0,0,17
"[470.55555555555554, 500.66666666666663)",0,0,0,0,3,7,8,0,0,18
"[500.66666666666663, 530.7777777777778)",0,0,0,0,1,5,4,4,1,15
"[530.7777777777778, 560.8888888888889)",0,0,0,0,0,0,4,2,3,9
"[560.8888888888889, 591]",0,0,0,0,0,0,1,1,1,3
Сумма,3,10,13,15,20,20,17,7,5,110


Выводы

4.
Исходя из результатов корреляционной таблицы вычислить значение выборочного коэффициента корреляции двумя способами: с помощью стандартной формулы и с помощью условных вариант. Убедиться, что результаты совпадают.

In [ ]:
v_x = np.array(ui_arr_nu)
v_y = np.array(ui_arr_e)

_rowsIndex = pd.Index([*v_x])
_colsIndex = pd.Index([*v_y])
cor2 = two_dimensional_variational_series.copy()
cor2 = cor2.drop(cor2.columns[-1], axis=1)
cor2 = cor2.drop(cor2.index[-1])
cor2.index = _rowsIndex
cor2.columns = _colsIndex

In [ ]:
cor2['Сумма'] = cor2.sum(axis=1)

In [ ]:
new_row = [cor2[col].sum() for col in cor2.columns]
cor2.loc['Сумма'] = new_row

cor2

,-4.014814814814815,-3.0111111111111115,-2.0074074074074075,-1.0037037037037042,0.0,1.003703703703703,2.0074074074074075,3.0111111111111115,4.014814814814813,Сумма
-5,1,2,0,0,0,0,0,0,0,3
-4,2,5,3,0,0,0,0,0,0,10
-3,0,3,7,5,1,0,0,0,0,16
-2,0,0,2,9,7,1,0,0,0,19
-1,0,0,1,1,8,7,0,0,0,17
0,0,0,0,0,3,7,8,0,0,18
1,0,0,0,0,1,5,4,4,1,15
2,0,0,0,0,0,0,4,2,3,9
3,0,0,0,0,0,0,1,1,1,3
Сумма,3,10,13,15,20,20,17,7,5,110


Вычисление выборочного коэффициента корреляции при помощи условных вариант

In [ ]:
cor3 = cor2.copy()

for i in range(len(v_x)):
    cor3.iloc[i,:-1] *= v_x[i]

for i in range(len(v_y)):
    cor3.iloc[:-1,i] *= v_y[i]

cor3.iloc[:,-1] = cor3.iloc[:,:-1].sum(axis=1)
cor3.iloc[-1,:] = cor3.iloc[:-1,:].sum(axis=0)
cor3

,-4.014814814814815,-3.0111111111111115,-2.0074074074074075,-1.0037037037037042,0.0,1.003703703703703,2.0074074074074075,3.0111111111111115,4.014814814814813,Сумма
-5,20.074074,30.111111,-0.000000,-0.000000,0,0.000000,0.000000,0.000000,0.000000,50.185185
-4,32.118519,60.222222,24.088889,-0.000000,0,0.000000,0.000000,0.000000,0.000000,116.429630
-3,-0.000000,27.100000,42.155556,15.055556,0,0.000000,0.000000,0.000000,0.000000,84.311111
-2,-0.000000,-0.000000,8.029630,18.066667,0,-2.007407,0.000000,0.000000,0.000000,24.088889
-1,-0.000000,-0.000000,2.007407,1.003704,0,-7.025926,0.000000,0.000000,0.000000,-4.014815
0,-0.000000,-0.000000,-0.000000,-0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.000000,-0.000000,-0.000000,-0.000000,0,5.018519,8.029630,12.044444,4.014815,29.107407
2,-0.000000,-0.000000,-0.000000,-0.000000,0,0.000000,16.059259,12.044444,24.088889,52.192593
3,-0.000000,-0.000000,-0.000000,-0.000000,0,0.000000,6.022222,9.033333,12.044444,27.100000
Сумма,52.192593,117.433333,76.281481,34.125926,0,-4.014815,30.111111,33.122222,40.148148,379.400000


In [ ]:
_sum = cor3["Сумма"]["Сумма"]

counts_x = np.array(list(cor2.iloc[:-1,-1]))
counts_y = np.array(list(cor2.iloc[-1,:-1]))

x_mean = sum(counts_x * v_x) / N
y_mean = sum(counts_y * v_y) / N

x_s = np.sqrt((sum(counts_x * v_x**2) / N - x_mean**2))
y_s = np.sqrt((sum(counts_y * v_y**2) / N - y_mean**2))

r_variants = (_sum - N * x_mean * y_mean) / (N * x_s * y_s)
r_variants

0.9045974082137044

Коэффициент корреляции близок к 1, это указывает на линейную зависимость между признаками.

Вычисление выборочного коэффициента корреляции стандартным способом

In [ ]:
def space_middle(space):
    return (space[1] - space[0]) / 2 + space[0]

middles_x = list(map(space_middle, interval_nu_series['Границы интервалов']))
middles_y = list(map(space_middle, interval_e_series['Границы интервалов']))
cor4 = cor2.copy()

for i in range(len(v_x)):
    cor4.iloc[i,:-1] *= middles_x[i]

for i in range(len(v_y)):
    cor4.iloc[:-1,i] *= middles_y[i]

cor4.iloc[:,-1] = cor4.iloc[:,:-1].sum(axis=1)
cor4.iloc[-1,:] = cor4.iloc[:-1,:].sum(axis=0)


In [ ]:
cor4.index = pd.Index([*middles_x, "Сумма"])
cor4.columns = pd.Index([*middles_y, "Сумма"])

In [ ]:
cor4

,78.62222222222222,90.66666666666666,102.71111111111111,114.75555555555555,126.8,138.84444444444443,150.88888888888889,162.93333333333334,174.97777777777776,Сумма
335.05555555555554,26342.812346,60756.740741,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,8.709955e+04
365.16666666666663,57420.429630,165542.222222,112520.022222,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,3.354827e+05
395.27777777777777,0.000000,107515.555556,284195.938272,226801.604938,5.012122e+04,0.000000e+00,0.000000e+00,0.000000,0.000000,6.686343e+05
425.3888888888889,0.000000,0.000000,87384.330864,439341.644444,3.775752e+05,5.906288e+04,0.000000e+00,0.000000,0.000000,9.633640e+05
455.5,0.000000,0.000000,46784.911111,52271.155556,4.620592e+05,4.427055e+05,0.000000e+00,0.000000,0.000000,1.003821e+06
485.6111111111111,0.000000,0.000000,0.000000,0.000000,1.847265e+05,4.719708e+05,5.861866e+05,0.000000,0.000000,1.242884e+06
515.7222222222222,0.000000,0.000000,0.000000,0.000000,6.539358e+04,3.580258e+05,3.112670e+05,336113.362963,90239.928395,1.161040e+06
545.8333333333334,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,3.294407e+05,177868.888889,286526.111111,7.938357e+05
575.9444444444445,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,8.690362e+04,93840.548148,100777.479012,2.815216e+05
Сумма,83763.241975,333814.518519,530885.202469,718414.404938,1.139876e+06,1.331765e+06,1.313798e+06,607822.800000,477543.518519,6.537682e+06


In [ ]:
_sum = cor4["Сумма"]["Сумма"]
r_standard = (_sum - _N * x_v2_nu * x_v2_e) / (_N * math.sqrt(sigma2_nu) * math.sqrt(sigma2_e))
r_standard

0.6892727148668081

Вычисленные r_variants и r_standard не совпадают. Данный результат, возможно, был вызван машинным округлением или числовой ошибкой.

5. Построение доверительного интервала

In [ ]:
from scipy.stats import norm


def trust_interval(g):
    q = 1 - g
    l = norm.ppf(1 - q/2)
    z = 0.5 * np.log((1 + r_variants) / (1 - r_variants))
    sigma_z = 1 / np.sqrt(N - 3)
    z_space = [z - l * sigma_z, z + l * sigma_z]
    r_space = np.tanh(z_space)
    print("Доверительный интервал для доверительной вероятности {}: \n\t{} < z < {}, \n\t{} < r < {}".format(g, *z_space, *r_space))

trust_interval(0.95)
trust_interval(0.99)

Доверительный интервал для доверительной вероятности 0.95: 
	1.3083624703035235 < z < 1.6855577149955807, 
	0.8638604850976889 < r < 0.9335790470551066
Доверительный интервал для доверительной вероятности 0.99: 
	1.249100802351138 < z < 1.7448193829479661, 
	0.8480312991579819 < r < 0.9407830373426416


6. Проверка статистической гипотезы при уровне значимости a=0.05

In [ ]:
from scipy.stats import t

a = 0.05
t_obs = r_variants * np.sqrt((N-2) / (1 - r_variants**2))
t_crit = t.ppf(1 - a/2, df=(N-2))
print("T_наблюдаемое {}".format(t_obs))
print("T_критическое {}".format(t_crit))

T_наблюдаемое 22.155773003166452
T_критическое 1.981967489688474


Так как T_наблюдаемое > T_критическое, гипотеза не принимается. Следовательно обнаруженная связь между признаками является случайной.

## Вывод
В данной практической работе был построен двумерный интервальный вариационный ряд, а также корреляционная таблица. Исходя из результатов таблицы были вычисленны значения выборочного коэффициента корреляции двумя способами: с помощью стандартной формулы и с помощью условных вариант. Вычисленные r_variants и r_standard не совпадают, скорее всего из-за машинного округления или числовой ошибки. Был построен доверительный интервал для коэффициента корреляции для уровней доверия 0.95, 0.99. Также осущественна проверка статистической гипотезы о равенстве коэффициента корреляции нулю.